## ST590 Project3-Group16
### Student: Autumn Biggie and YiHua Lin


## 1. Introduction: 

The S&P 500 is a capitalization-weighted index of the top 500 publicly traded companies in the United States (top 500 meaning the companies with the largest market cap). The S&P 500 index is a useful index to study because it generally reflects the health of the overall U.S. stock market.

We are going to use S&P 500 Financial Dataset to predict price from Kraggle website and to determine the high-quality company

## Predictor Introduction( 4 character variables and 10 numeric variables): 

character variables:

**Symbol-** Ticker symbol used to uniquely identify each company on a particular stock market

**Name-** Legal name of the company

**Sector-** An area of the economy where businesses share a related product or service

**SEC Filings-** Helpful documents relating to a company

numeric variables:

**Price/Earnings(P/E)-** financial indicator equal to the ratio of the market value of a share to the annual earnings per share

**Dividend yield-** the ratio of the annual dividend per share to the share price. This value is most often expressed as a percentage.

**Earnings/Share(EPS) -** financial indicator equal to the ratio of the company's net profit available for distribution to the average annual number of ordinary shares.

**EBITDA-** an analytical indicator equal to the amount of profit before deduction of expenses on payment of interest, taxes, depreciation and amortization.Often used as a proxy for its profitability

**Price/Sales(P/S)-** financial indicator equal to the ratio of a company's market capitalization to its annual revenue.

**Price/book(P/B)-** financial ratio equal to the ratio of the company's current market capitalization to its book value.

**Price-** Price per share of the company

**52 week high-** The annual high of a company’s share price

**52 week low-** The annual low of a company’s share price
        - 
        
**Market Cap-** The market value of a company’s shares (calculated as share price x number of shares)


## Import appropriate modules for starting spark session, using pandas on spark, and accessing math functions

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import scipy.stats as stats
import sqlite3
import pylab as pl
import plotly
import pyspark.pandas as ps
import plotly.graph_objs as go
import seaborn as sns
import sys
import os
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
from plotly.offline import init_notebook_mode, iplot
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.functions import isnan, when, count, col
spark = SparkSession.builder.getOrCreate()
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from pyspark.ml.feature import SQLTransformer
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml.regression import LinearRegression
from pyspark.mllib.tree import RandomForest
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import DecisionTreeRegressor

## 2. Supervised Learning and Data Split

## My goal is predict short-term volatility for stocks across different sectors

In [2]:
# Read in S&P 500 data set using pandas
financials = pd.read_csv("financials.csv") 
financials.head()

,Symbol,Name,Sector,Price,Price/Earnings,Dividend Yield,Earnings/Share,52 Week Low,52 Week High,Market Cap,EBITDA,Price/Sales,Price/Book,SEC Filings
0,MMM,3M Company,Industrials,222.89,24.31,2.332862,7.92,259.77,175.490,1.387210e+11,9.048000e+09,4.390271,11.34,http://www.sec.gov/cgi-bin/browse-edgar?action...
1,AOS,A.O. Smith Corp,Industrials,60.24,27.76,1.147959,1.70,68.39,48.925,1.078342e+10,6.010000e+08,3.575483,6.35,http://www.sec.gov/cgi-bin/browse-edgar?action...
2,ABT,Abbott Laboratories,Health Care,56.27,22.51,1.908982,0.26,64.60,42.280,1.021210e+11,5.744000e+09,3.740480,3.19,http://www.sec.gov/cgi-bin/browse-edgar?action...
3,ABBV,AbbVie Inc.,Health Care,108.48,19.41,2.499560,3.29,125.86,60.050,1.813860e+11,1.031000e+10,6.291571,26.14,http://www.sec.gov/cgi-bin/browse-edgar?action...
4,ACN,Accenture plc,Information Technology,150.51,25.47,1.714470,5.44,162.60,114.820,9.876586e+10,5.643228e+09,2.604117,10.62,http://www.sec.gov/cgi-bin/browse-edgar?action...


In [3]:
# Create Dummy variable for Sector
financialsj=pd.get_dummies(financials["Sector"])
#join dummy dataframe to pandas dataframe
financials=financials.join(financialsj)
financials.head()

,Symbol,Name,Sector,Price,Price/Earnings,Dividend Yield,Earnings/Share,52 Week Low,52 Week High,Market Cap,...,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Telecommunication Services,Utilities
0,MMM,3M Company,Industrials,222.89,24.31,2.332862,7.92,259.77,175.490,1.387210e+11,...,0,0,0,0,1,0,0,0,0,0
1,AOS,A.O. Smith Corp,Industrials,60.24,27.76,1.147959,1.70,68.39,48.925,1.078342e+10,...,0,0,0,0,1,0,0,0,0,0
2,ABT,Abbott Laboratories,Health Care,56.27,22.51,1.908982,0.26,64.60,42.280,1.021210e+11,...,0,0,0,1,0,0,0,0,0,0
3,ABBV,AbbVie Inc.,Health Care,108.48,19.41,2.499560,3.29,125.86,60.050,1.813860e+11,...,0,0,0,1,0,0,0,0,0,0
4,ACN,Accenture plc,Information Technology,150.51,25.47,1.714470,5.44,162.60,114.820,9.876586e+10,...,0,0,0,0,0,1,0,0,0,0


In [4]:
findata = spark.createDataFrame(financials) #convert to spark data frame
findata.show()

+------+--------------------+--------------------+------+--------------+--------------+--------------+-----------+------------+---------------+----------+-----------+----------+--------------------+----------------------+----------------+------+----------+-----------+-----------+----------------------+---------+-----------+--------------------------+---------+
|Symbol|                Name|              Sector| Price|Price/Earnings|Dividend Yield|Earnings/Share|52 Week Low|52 Week High|     Market Cap|    EBITDA|Price/Sales|Price/Book|         SEC Filings|Consumer Discretionary|Consumer Staples|Energy|Financials|Health Care|Industrials|Information Technology|Materials|Real Estate|Telecommunication Services|Utilities|
+------+--------------------+--------------------+------+--------------+--------------+--------------+-----------+------------+---------------+----------+-----------+----------+--------------------+----------------------+----------------+------+----------+-----------+------

In [5]:
#change the column name for train set and test set for easier EDA
findata = findata.select([func.col(col).alias(col.replace('/', '_')) for col in findata.columns])
findata = findata.select([func.col(col).alias(col.replace(' ', '_')) for col in findata.columns])

In [6]:
findata=findata.filter(findata.Price<400)

In [7]:
#remove the missing data
findata=findata.na.drop()

In [8]:
#split pandas dataframe then transfer train and test set to spark
# Create training and test set with 80%/20% split, returning #obs in each set
# We will use THESE sets during modeling
train, test = findata.randomSplit([0.8, 0.2], seed = 3)
print(train.count(), test.count())

384 103


## 3. EDA

## Outlier

### Plot- Matplotlib and Seaborn

### 4. Modeling

## Model 1

In [9]:
#stock price prediction 
sqlTrans = SQLTransformer(
    statement =  "Select Dividend_Yield, Market_Cap/1000000000 as Market_Cap, EBITDA/1000000000 as EBITDA, Price_Sales, Price_Book, 52_Week_Low, Price as label FROM __THIS__;"
)

In [10]:
sqlTrans.transform(train).show(5)

+--------------+------------+--------+-----------+----------+-----------+------+
|Dividend_Yield|  Market_Cap|  EBITDA|Price_Sales|Price_Book|52_Week_Low| label|
+--------------+------------+--------+-----------+----------+-----------+------+
|     0.8756979|21.984606918|   1.094|   6.493563|      4.56|       75.0| 65.05|
|     0.7782101|24.594852352|   5.761| 0.58022565|      6.03|      59.08|  48.6|
|     1.5795412|     809.508|  79.386|  3.4586093|      5.66|      180.1|155.15|
|     2.4995599|     181.386|   10.31|   6.291571|     26.14|     125.86|108.48|
|     1.6132456|20.587704101|0.991884| 0.17396984|      9.73|     106.27| 91.55|
+--------------+------------+--------+-----------+----------+-----------+------+
only showing top 5 rows



In [11]:
#put 6 predictors into feature
assembler = VectorAssembler(inputCols = ["Dividend_Yield", "Market_Cap", "EBITDA", "Price_Sales", "Price_Book", "52_Week_Low"], outputCol = "features", handleInvalid = 'keep')


In [12]:
# Use CV to tune the parameter

CVTrainModel1 = LinearRegression()
pipeline = Pipeline(stages = [sqlTrans, assembler, CVTrainModel1])

In [13]:
#Grid the parameter
paramGrid = ParamGridBuilder() \
    .addGrid(CVTrainModel1.regParam, [0, 0.01, 0.05]) \
    .addGrid(CVTrainModel1.fitIntercept, [False, True]) \
    .addGrid(CVTrainModel1.elasticNetParam, [0, 0.05, 0.1]) \
    .build()

In [14]:
# Use CV to evaluate the RMSE to find min RMSE ( find the best model from 18 models)
crossval = CrossValidator(estimator = pipeline,
                          estimatorParamMaps = paramGrid,
                          evaluator = RegressionEvaluator(metricName='rmse'),
                          numFolds=5)

In [15]:
# put training set to fit the model
CVModel = crossval.fit(train)

In [16]:
# 18 models output
list(zip(CVModel.avgMetrics, paramGrid))

[(14.051417349377228,
  {Param(parent='LinearRegression_ee19c0f3f787', name='regParam', doc='regularization parameter (>= 0).'): 0.0,
   Param(parent='LinearRegression_ee19c0f3f787', name='fitIntercept', doc='whether to fit an intercept term.'): False,
   Param(parent='LinearRegression_ee19c0f3f787', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0}),
 (14.051417349377228,
  {Param(parent='LinearRegression_ee19c0f3f787', name='regParam', doc='regularization parameter (>= 0).'): 0.0,
   Param(parent='LinearRegression_ee19c0f3f787', name='fitIntercept', doc='whether to fit an intercept term.'): False,
   Param(parent='LinearRegression_ee19c0f3f787', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.05}),
 (14.051417349377228,
  {Param(parent='LinearRegression

In [17]:
# use the best model to predict
prediction=CVModel.transform(test)

In [18]:
prediction.show()

+--------------+------------+--------+-----------+----------+-----------+------+--------------------+------------------+
|Dividend_Yield|  Market_Cap|  EBITDA|Price_Sales|Price_Book|52_Week_Low| label|            features|        prediction|
+--------------+------------+--------+-----------+----------+-----------+------+--------------------+------------------+
|    0.21832074| 8.123611867|0.853941|  1.1301061|      2.51|     169.55|109.63|[0.21832074,8.123...|  139.701744308403|
|     1.9089824|     102.121|   5.744|  3.7404804|      3.19|       64.6| 56.27|[1.9089824,102.12...|56.760769778498585|
|     2.2190912|50.337702249|  2.7674|  4.0483294|     12.86|     125.24|108.25|[2.2190912,50.337...|105.13868506828496|
|     0.9240122|13.632608582|  2.1432|  1.7282298|      8.28|     278.33| 240.6|[0.9240122,13.632...|229.30002894040618|
|           0.0|24.348294504| -0.3781|   16.50682|    224.13|      131.1|104.81|[0.0,24.348294504...|110.32147731565992|
|     2.4299066|   33.422948|   

In [19]:
test_error = RegressionEvaluator().evaluate(prediction)
print(test_error)

8.916534849860872


## Model 2

In [20]:
#Random forest
rf_assembler = VectorAssembler(inputCols = [ "Dividend_Yield", "Market_Cap", "EBITDA", "Price_Sales", "Price_Book", "52_Week_Low"], outputCol = "features")

In [21]:
# Train a RandomForest model.
rf = RandomForestRegressor(featuresCol="features")

In [22]:
pipeline = Pipeline(stages=[rf_assembler, rf])

In [23]:
paramGrid = (ParamGridBuilder()
            .addGrid(rf.maxDepth, [2, 5, 10])
            .addGrid(rf.maxBins, [5, 10, 20])
            .build())

In [24]:
CV = CrossValidator(estimator = pipeline,
                    estimatorParamMaps = paramGrid,
                    evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse"),
                    numFolds = 5)

In [25]:
rf_train= rf_assembler.transform(sqlTrans.transform(train)).select("Dividend_Yield", "Market_Cap", "EBITDA", "Price_Sales", "Price_Book", "52_Week_Low","label")

In [26]:
rf_train.show(5)
model = CV.fit(rf_train)

+--------------+------------+--------+-----------+----------+-----------+------+
|Dividend_Yield|  Market_Cap|  EBITDA|Price_Sales|Price_Book|52_Week_Low| label|
+--------------+------------+--------+-----------+----------+-----------+------+
|     0.8756979|21.984606918|   1.094|   6.493563|      4.56|       75.0| 65.05|
|     0.7782101|24.594852352|   5.761| 0.58022565|      6.03|      59.08|  48.6|
|     1.5795412|     809.508|  79.386|  3.4586093|      5.66|      180.1|155.15|
|     2.4995599|     181.386|   10.31|   6.291571|     26.14|     125.86|108.48|
|     1.6132456|20.587704101|0.991884| 0.17396984|      9.73|     106.27| 91.55|
+--------------+------------+--------+-----------+----------+-----------+------+
only showing top 5 rows



In [27]:
# Make predictions.
rf_test= rf_assembler.transform(sqlTrans.transform(test)).select("Dividend_Yield", "Market_Cap", "EBITDA", "Price_Sales", "Price_Book", "52_Week_Low","label")
predictions = model.transform(rf_test)

In [28]:
# Select (prediction, true label) and compute test error
evaluator=RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = RegressionEvaluator().evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
predictions.show()

Root Mean Squared Error (RMSE) on test data = 15.3208
+--------------+------------+--------+-----------+----------+-----------+------+--------------------+------------------+
|Dividend_Yield|  Market_Cap|  EBITDA|Price_Sales|Price_Book|52_Week_Low| label|            features|        prediction|
+--------------+------------+--------+-----------+----------+-----------+------+--------------------+------------------+
|    0.21832074| 8.123611867|0.853941|  1.1301061|      2.51|     169.55|109.63|[0.21832074,8.123...|          163.5665|
|     1.9089824|     102.121|   5.744|  3.7404804|      3.19|       64.6| 56.27|[1.9089824,102.12...| 56.72638571428572|
|     2.2190912|50.337702249|  2.7674|  4.0483294|     12.86|     125.24|108.25|[2.2190912,50.337...|114.46683333333333|
|     0.9240122|13.632608582|  2.1432|  1.7282298|      8.28|     278.33| 240.6|[0.9240122,13.632...|206.67827941176466|
|           0.0|24.348294504| -0.3781|   16.50682|    224.13|      131.1|104.81|[0.0,24.348294504..

## Model 3

#Decision tree


In [29]:
#df
dt_assembler = VectorAssembler(inputCols = [ "Dividend_Yield", "Market_Cap", "EBITDA", "Price_Sales", "Price_Book", "52_Week_Low"], outputCol = "features")
# Train a RandomForest model.
dt = DecisionTreeRegressor(featuresCol="features")
pipeline = Pipeline(stages=[dt_assembler, dt])
paramGrid = (ParamGridBuilder()
            .addGrid(dt.maxDepth, [2, 5, 10])
            .addGrid(dt.maxBins, [5, 10, 20])
            .build())
CV = CrossValidator(estimator = pipeline,
                    estimatorParamMaps = paramGrid,
                    evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse"),
                    numFolds = 5)
dt_train= dt_assembler.transform(sqlTrans.transform(train)).select("Dividend_Yield", "Market_Cap", "EBITDA", "Price_Sales", "Price_Book", "52_Week_Low","label")
model = CV.fit(dt_train)
# Make predictions.
dt_test= dt_assembler.transform(sqlTrans.transform(test)).select("Dividend_Yield", "Market_Cap", "EBITDA", "Price_Sales", "Price_Book", "52_Week_Low","label")
predictions = model.transform(dt_test)
# Select (prediction, true label) and compute test error
evaluator=RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = RegressionEvaluator().evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
predictions.show()

Root Mean Squared Error (RMSE) on test data = 31.5583
+--------------+------------+--------+-----------+----------+-----------+------+--------------------+------------------+
|Dividend_Yield|  Market_Cap|  EBITDA|Price_Sales|Price_Book|52_Week_Low| label|            features|        prediction|
+--------------+------------+--------+-----------+----------+-----------+------+--------------------+------------------+
|    0.21832074| 8.123611867|0.853941|  1.1301061|      2.51|     169.55|109.63|[0.21832074,8.123...|158.17650000000003|
|     1.9089824|     102.121|   5.744|  3.7404804|      3.19|       64.6| 56.27|[1.9089824,102.12...| 59.58698113207547|
|     2.2190912|50.337702249|  2.7674|  4.0483294|     12.86|     125.24|108.25|[2.2190912,50.337...|           116.648|
|     0.9240122|13.632608582|  2.1432|  1.7282298|      8.28|     278.33| 240.6|[0.9240122,13.632...|            387.65|
|           0.0|24.348294504| -0.3781|   16.50682|    224.13|      131.1|104.81|[0.0,24.348294504..